In [2]:
from create_spark import init_local_session, init_remote_session

/spark


In [1]:
import socket
import sys
import os
from pyspark import SparkContext, SparkConf
from pyspark.streaming import StreamingContext
from pyspark.sql import Row, SparkSession
from os.path import abspath
import findspark
import time 

In [ ]:
# # 스파크 생성 
# #spark = init_local_session()
# spark = SparkSession.builder.master('yarn').appName('SPARK_APP_NAME')\
#     .config("spark.sql.warehouse.dir", '/user/hive/warehouse')\
#     .config("spark.security.credentials.hiveserver2.enabled", 'false')\
#     .config("spark.sql.hive.hiveserver2.jdbc.url", 'jdbc:hive2://dn01:10000/metastore;user=hive;password=hive')\
#     .enableHiveSupport().getOrCreate()
#     #.config("spark.datasource.hive.warehouse.metastoreUri", 'thrift://localhost:9083')\
# # spark = init_remote_session(enable_hive=True)
# sc = spark.sparkContext
# sc

In [5]:
# # 스파크 생성 
# spark = SparkSession.builder.master('yarn').appName('SPARK_APP_NAME')\
#     .config("spark.sql.warehouse.dir", '/user/hive/warehouse')\
#     .getOrCreate()
#     #.config("spark.datasource.hive.warehouse.metastoreUri", 'thrift://localhost:9083')\
spark = init_remote_session()
sc = spark.sparkContext
sc

/usr/local/lib/python3.6/dist-packages/pyspark/context.py:238: FutureWarning: Python 3.6 support is deprecated in Spark 3.2.
  FutureWarning


<SparkContext master=yarn appName=DATA-Preparation>

In [7]:
# spark.read.text("hdfs://namenode:9000/user/hive/warehouse/testdb.db/intbl").show()

In [6]:
#spark.sql("CREATE TABLE IF NOT EXISTS src (key INT, value STRING) USING hive") # Hive support is required
# spark.sql("CREATE TABLE IF NOT EXISTS src (key INT, value STRING) ")

In [8]:
# spark.sql("select * from testdb.intbl;").show()

In [9]:
# spark.read.text("hdfs://namenode:9000/user/hive/warehouse/testdb.db/intbl").show()

In [10]:
# Define path
# DATA_ROOT = f'{os.getcwd()}/data'
DATA_PATH = '/user/root'

In [11]:
# channel, day, seg, setop, 1000
# FILE_FORMAT = 'parquet'
FILE_FORMAT = 'csv'

# setop data n 개 만들기
def create_setops():
    setop_count = 10000
    setop_name = ['ST_A', 'ST_B', 'ST_C', 'ST_D', 'ST_E']
    setops = []
    for s in setop_name:
        for i in range(0, int(setop_count/len(setop_name))):
            setops.append(f'{s}_{i:03d}')
            
    print(setops[-10:])
    return setops

# channel, day, seg data 생성 
def create_others():
    # 20
    channels = ['KBS', 'MBC', 'SBS', 'JTBC', 'CBS' ,  'OCN', 'TVN', 'TVCH', 'BTN', 'EBS',  'Arirang', 'JTV', 'GAME-TV', 'HBC', 'BBC',  'CNN', 'CNBC', 'CCN', 'NHK', 'ABC'] 
    days = ['mon', 'tue', 'wed', 'thu', 'fri', 'sat', 'sun']
    hour_bands = ['00', '01', '10', '11', '23']
    segs = ['Agriculture', 'Game']
    rows = []
    row = []
    for c in channels:
        for d in days:
            for s in segs:
                row = [c, d, s]
                rows.append(row)
                
    print(rows[:5])
    return rows

def merge_to_inventory(setops, rows):
    inven_time = 10000 # 하드코딩 시간
    invens = []
    for r in rows:
        for s in setops:
            invens.append(r + [s, inven_time])
    print(f'Inventory Length : {len(invens):,}')
    return invens

def define_schema():
    from pyspark.sql.types import StructType, StructField, StringType, LongType
    columns = [
        StructField("channel", StringType())
        , StructField("day", StringType())
        , StructField("seg", StringType())
        , StructField("setop", StringType())
        , StructField("remains", LongType())
    ]
    inven_schema = StructType(columns)
    return inven_schema

def save_inventory(invens, spark_session=spark, file_name=f'{DATA_PATH}/inven', sample_count=10000):
    inven_schema = define_schema()
    if (sample_count <= 0):
        # all data 
        rdd = spark_session.sparkContext.parallelize(invens)
    else:
        # sampling data
        rdd = spark_session.sparkContext.parallelize(invens[:sample_count])
    df = spark_session.createDataFrame(rdd, inven_schema)
    #df.write.save(path=file_name, format='csv', mode='append', sep=',')
    df.write.save(path=file_name, format=FILE_FORMAT, mode='append', sep=',')
        

In [12]:
setops = create_setops()
rows = create_others()

['ST_E_1990', 'ST_E_1991', 'ST_E_1992', 'ST_E_1993', 'ST_E_1994', 'ST_E_1995', 'ST_E_1996', 'ST_E_1997', 'ST_E_1998', 'ST_E_1999']
[['KBS', 'mon', 'Agriculture'], ['KBS', 'mon', 'Game'], ['KBS', 'tue', 'Agriculture'], ['KBS', 'tue', 'Game'], ['KBS', 'wed', 'Agriculture']]


In [13]:
%%time
invens = merge_to_inventory(setops, rows)

Inventory Length : 2,800,000
CPU times: user 1.74 s, sys: 229 ms, total: 1.97 s
Wall time: 1.97 s


In [15]:
%%time
# invens data 파일로 저장하기 
# gcp vm local : 12 s. csv - 90 MB, parq - 4.5 MB, 280만 rows 
SAMPLING_COUNT = int(1e7)
TABLE_NAME = "hdfs://namenode:9000/user/root/inven/inv"
# need file permission  
#TABLE_NAME = f'file:///home'

print(f'SAMPLING_COUNT : {SAMPLING_COUNT:,}')
print(f'TABLE_NAME : {TABLE_NAME}')
print(f'Inventory Length : {len(invens):,}')
      
save_inventory(invens, spark, TABLE_NAME, SAMPLING_COUNT)

SAMPLING_COUNT : 10,000,000
TABLE_NAME : hdfs://namenode:9000/user/root/inven/inv
Inventory Length : 2,800,000
CPU times: user 1.09 s, sys: 91.2 ms, total: 1.18 s
Wall time: 10.5 s


In [16]:
%%time 
# 저장 결과 확인하기 
lines = spark.read.format(FILE_FORMAT).schema(define_schema()).option('path', TABLE_NAME).load()
data_count = lines.count()
print(f'DATA Count : {data_count:,}')
lines.show(5)

DATA Count : 2,800,010
+-------+---+----+---------+-------+
|channel|day| seg|    setop|remains|
+-------+---+----+---------+-------+
|    EBS|sun|Game|ST_E_1808|  10000|
|    EBS|sun|Game|ST_E_1809|  10000|
|    EBS|sun|Game|ST_E_1810|  10000|
|    EBS|sun|Game|ST_E_1811|  10000|
|    EBS|sun|Game|ST_E_1812|  10000|
+-------+---+----+---------+-------+
only showing top 5 rows

CPU times: user 3.28 ms, sys: 3.64 ms, total: 6.92 ms
Wall time: 2.61 s


In [17]:
%%time
# ch 별로 setop 번호 순서 5개씩 보여주기
# csv : 3.8 s, parq : 2.4s
from pyspark.sql.window import Window
from pyspark.sql.functions import col, row_number 

windowPart = Window.partitionBy('channel').orderBy(col('setop').desc())
lines = spark.read.format(FILE_FORMAT).schema(define_schema()).option('path', TABLE_NAME).load()
df2 = lines.withColumn('row', row_number().over(windowPart)) .filter(col('row') <= 5)
df2.show(50)

+-------+---+-----------+--------+-------+---+
|channel|day|        seg|   setop|remains|row|
+-------+---+-----------+--------+-------+---+
|    NHK|mon|Agriculture|ST_E_999|  10000|  1|
|    NHK|mon|       Game|ST_E_999|  10000|  2|
|    NHK|tue|Agriculture|ST_E_999|  10000|  3|
|    NHK|tue|       Game|ST_E_999|  10000|  4|
|    NHK|wed|Agriculture|ST_E_999|  10000|  5|
|    EBS|mon|Agriculture|ST_E_999|  10000|  1|
|    EBS|mon|       Game|ST_E_999|  10000|  2|
|    EBS|tue|Agriculture|ST_E_999|  10000|  3|
|    EBS|tue|       Game|ST_E_999|  10000|  4|
|    EBS|wed|Agriculture|ST_E_999|  10000|  5|
|   TVCH|mon|Agriculture|ST_E_999|  10000|  1|
|   TVCH|mon|       Game|ST_E_999|  10000|  2|
|   TVCH|tue|Agriculture|ST_E_999|  10000|  3|
|   TVCH|tue|       Game|ST_E_999|  10000|  4|
|   TVCH|wed|Agriculture|ST_E_999|  10000|  5|
|    CCN|mon|Agriculture|ST_E_999|  10000|  1|
|    CCN|mon|       Game|ST_E_999|  10000|  2|
|    CCN|tue|Agriculture|ST_E_999|  10000|  3|
|    CCN|tue|

In [ ]:
# rdb 



In [18]:
spark.stop()